In [4]:
import numpy as np
from scipy.io import wavfile
import os
import pygame
import piano_lists as pl #피아노 음표 및 관련 데이터를 제공
from pygame import mixer #오디오 믹싱과 관련된 함수를 제공

# 가우시안 분포 함수 정의
def gaussian_waveform(x, mu, sigma):
    return np.exp(-(x - mu)**2 / (2 * sigma**2))

# 각 음에 대한 파라미터 설정
notes_params = {
    'C4': {'L': 0.657, 'c': 329.56, 'k': 1.25, 'b1': 1.11, 'b2': 2.7e-04, 'f0': 262.22},
    'Db4': {'L': 0.622, 'c': 330.60, 'k': 1.27, 'b1': 1.18, 'b2': 2.8e-04, 'f0': 277.81},
    'D4': {'L': 0.590, 'c': 332.32, 'k': 1.29, 'b1': 1.25, 'b2': 3.0e-04, 'f0': 294.34},
    'Eb4': {'L': 0.559, 'c': 333.61, 'k': 1.31, 'b1': 1.33, 'b2': 3.22e-04, 'f0': 311.82},
    'E4': {'L': 0.529, 'c': 334.52, 'k': 1.33, 'b1': 1.41, 'b2': 3.4e-04, 'f0': 330.36},
    'F4': {'L': 0.501, 'c': 335.75, 'k': 1.35, 'b1': 1.50, 'b2': 3.6e-04, 'f0': 350.05},
    'Gb4': {'L': 0.475, 'c': 336.76, 'k': 1.37, 'b1': 1.59, 'b2': 3.81e-04, 'f0': 370.80},
    'G4': {'L': 0.450, 'c': 337.58, 'k': 1.39, 'b1': 1.69, 'b2': 4.03e-04, 'f0': 392.87},
    'Ab4': {'L': 0.426, 'c': 340.65, 'k': 1.41, 'b1': 1.79, 'b2': 4.26e-04, 'f0': 416.26},
    'A4': {'L': 0.404, 'c': 341.33, 'k': 1.43, 'b1': 1.9, 'b2': 4.51e-04, 'f0': 441.00},
    'Bb4': {'L': 0.383, 'c': 343.87, 'k': 1.45, 'b1': 2.01, 'b2': 4.77e-04, 'f0': 467.19},
    'B4': {'L': 0.363, 'c': 345.34, 'k': 1.47, 'b1': 2.13, 'b2': 5.05e-04, 'f0': 494.96},
    'C5': {'L': 0.344, 'c': 346.84, 'k': 1.49, 'b1': 2.26, 'b2': 5.34e-04, 'f0': 524.48},
    'Db5': {'L': 0.325, 'c': 348.00, 'k': 1.50, 'b1': 2.35, 'b2': 5.6e-04, 'f0': 554.37}
}

nseg = 50  # 공간 분할 개수
Fs = 44100*3  # 샘플링 주파수

for note, p in notes_params.items():
 wav_filename = rf'{note}.wav'
 if not os.path.exists(wav_filename):
    L = p['L']
    c = p['c']
    k = p['k']
    b1 = p['b1']
    b2 = p['b2']
    f0 = p['f0']  # 음높이에 따른 초기 진폭 계산

    X = L / nseg
    Nx = int(L / X) + 1
    mu = L / 2  # 평균 설정 (중간 값)
    sigma = L / 10  # 표준 편차 설정
    initial_waveform = gaussian_waveform(np.linspace(0, L, Nx), mu, sigma)

    numerator = -4 * b2 + np.sqrt(16 * b2 ** 2 + 4 * (c ** 2 * X ** 2 + 4 * k ** 2))
    denominator = 2 * (c ** 2 * X ** 2 + 4 * k ** 2)
    max_T = X ** 2 * numerator / denominator

    T = min(1 / Fs, max_T)  # 시간 간격을 샘플링 주파수와 계산된 최대 안정 간격 중 작은 값으로 설정

    Nx = int(L / X) + 1
    Nt = int(1.0 / T) + 1

    y = np.zeros((Nx, Nt), dtype=np.float64)
    y[:, 0] = initial_waveform  # 초기 조건 설정에 가우시안 파형 적용
    # 차분방정식 계수 계산
    lambda_ = c * T / X
    mu = k * T / X ** 2
    a10 = (2 - 2 * lambda_ ** 2 - 6 * mu ** 2 - 4 * b2 * mu * k) / (1 + b1 * T)
    a11 = (lambda_ ** 2 + 4 * mu ** 2 + 2 * b2 * mu * k) / (1 + b1 * T)
    a12 = -mu ** 2 / (1 + b1 * T)
    a20 = (-1 + 4 * b2 * mu * k + b1 * T) / (1 + b1 * T)
    a21 = (-2 * b2 * mu * k) / (1 + b1 * T)

    y[0, :] = 0 # t = 0 일 때, 모든 m 에 대해 y 값이 0
    y[-1, :] = 0  # t = -1 일 때, 모든 m 에 대해 y 값이 0
    # 차분방정식 계산
    for n in range(1, Nt - 1):
        for m in range(2, Nx - 2):
            y[m, n + 1] = (a10 * y[m, n] + a11 * (y[m + 1, n] + y[m - 1, n]) +
                           a12 * (y[m + 2, n] + y[m - 2, n]) + a20 * y[m, n - 1] +
                           a21 * (y[m + 1, n - 1] + y[m - 1, n - 1]))

    # WAV 파일 저장
    y_max = np.max(np.abs(y))
    audio_data = (y / y_max * 32767).astype(np.int16)
    wavfile.write(rf'{note}.wav', Fs, audio_data[int(Nx / 2), :])
    print(f"WAV 파일 {note}이 생성되었습니다.")

#피아노 생성하기
pygame.init()
pygame.mixer.set_num_channels(50)

font = pygame.font.SysFont('arial', 48)
medium_font = pygame.font.SysFont('arial', 28)
small_font = pygame.font.SysFont('arial', 16)
real_small_font = pygame.font.SysFont('arial', 10)
fps = 60
timer = pygame.time.Clock()
WIDTH = 8 * 35
HEIGHT = 400
screen = pygame.display.set_mode([WIDTH, HEIGHT])
active_whites = []
active_blacks = []
left_oct = 4

left_hand = pl.left_hand
piano_notes = pl.piano_notes
white_notes = pl.white_notes
black_notes = pl.black_notes
black_labels = pl.black_labels
white_sounds = [mixer.Sound(f'{note}.wav') for note in white_notes]
black_sounds = [mixer.Sound(f'{note}.wav') for note in black_notes]
for sound in white_sounds + black_sounds:
    sound.set_volume(0.5)
# 건반에 대한 소리 불러오기
for i in range(len(white_notes)):
    white_sounds.append(mixer.Sound(f'{white_notes[i]}.wav'))

for i in range(len(black_notes)):
    black_sounds.append(mixer.Sound(f'{black_notes[i]}.wav'))

pygame.display.set_caption("Python Piano")

#피아노 그리기
def draw_piano(whites, blacks):
    white_rects = []
    for i in range(8):
        rect = pygame.draw.rect(screen, 'white', [i * 35, HEIGHT - 300, 35, 300], 0, 2)
        white_rects.append(rect)
        pygame.draw.rect(screen, 'black', [i * 35, HEIGHT - 300, 35, 300], 2, 2)
        key_label = small_font.render(white_notes[i], True, 'black')
        screen.blit(key_label, (i * 35 + 3, HEIGHT - 20))
    skip_count = 0
    last_skip = 2
    skip_track = 2
    black_rects = []
    for i in range(6):
        rect = pygame.draw.rect(screen, 'black', [23 + (i * 35) + (skip_count * 35), HEIGHT - 300, 24, 200], 0, 2)
        for q in range(len(blacks)):
            if blacks[q][0] == i:
                if blacks[q][1] > 0:
                    pygame.draw.rect(screen, 'green', [23 + (i * 35) + (skip_count * 35), HEIGHT - 300, 24, 200], 2, 2)
                    blacks[q][1] -= 1

        key_label = real_small_font.render(black_labels[i], True, 'white')
        screen.blit(key_label, (25 + (i * 35) + (skip_count * 35), HEIGHT - 120))
        black_rects.append(rect)
        skip_track += 1
        if last_skip == 2 and skip_track == 4:
            last_skip = 4
            skip_track = 0
            skip_count += 1
        elif last_skip == 4 and skip_track == 3:
            last_skip = 2
            skip_track = 0
            skip_count += 1
    #활성화된 키 강조하기
    for i in range(len(whites)):
        if whites[i][1] > 0:
            j = whites[i][0]
            pygame.draw.rect(screen, 'green', [j * 35, HEIGHT - 100, 35, 100], 2, 2)
            whites[i][1] -= 1

    return white_rects, black_rects, whites, blacks


def draw_hands(leftOct, leftHand):
    # left hand
    pygame.draw.rect(screen, 'dark gray', [(leftOct * 35) - 140, HEIGHT - 60, 280, 30], 0, 4)
    pygame.draw.rect(screen, 'black', [(leftOct * 35) - 140, HEIGHT - 60, 280, 30], 4, 4)
    text = small_font.render(leftHand[0], True, 'white')
    screen.blit(text, ((leftOct * 35) - 130, HEIGHT - 55))
    text = small_font.render(leftHand[2], True, 'white')
    screen.blit(text, ((leftOct * 35) - 95, HEIGHT - 55))
    text = small_font.render(leftHand[4], True, 'white')
    screen.blit(text, ((leftOct * 35) - 60, HEIGHT - 55))
    text = small_font.render(leftHand[5], True, 'white')
    screen.blit(text, ((leftOct * 35) - 25, HEIGHT - 55))
    text = small_font.render(leftHand[7], True, 'white')
    screen.blit(text, ((leftOct * 35) + 10, HEIGHT - 55))
    text = small_font.render(leftHand[9], True, 'white')
    screen.blit(text, ((leftOct * 35) + 45, HEIGHT - 55))
    text = small_font.render(leftHand[11], True, 'white')
    screen.blit(text, ((leftOct * 35) + 80, HEIGHT - 55))
    text = small_font.render(leftHand[12], True, 'white')
    screen.blit(text, ((leftOct * 35) + 115, HEIGHT - 55))
    text = small_font.render(leftHand[1], True, 'black')
    screen.blit(text, ((leftOct * 35) - 113, HEIGHT - 55))
    text = small_font.render(leftHand[3], True, 'black')
    screen.blit(text, ((leftOct * 35) - 78, HEIGHT - 55))
    text = small_font.render(leftHand[6], True, 'black')
    screen.blit(text, ((leftOct * 35) - 8, HEIGHT - 55))
    text = small_font.render(leftHand[8], True, 'black')
    screen.blit(text, ((leftOct * 35) + 27, HEIGHT - 55))
    text = small_font.render(leftHand[10], True, 'black')
    screen.blit(text, ((leftOct * 35) + 62, HEIGHT - 55))


def draw_title_bar():
    title_text = font.render('Play piano!', True, 'white')
    screen.blit(title_text, (0, 18))
    title_text = font.render('Play piano!', True, 'black')
    screen.blit(title_text, (2, 20))


run = True
while run:
    left_dict = {'Z': f'C{left_oct}',
                 'S': f'C#{left_oct}',
                 'X': f'D{left_oct}',
                 'D': f'D#{left_oct}',
                 'C': f'E{left_oct}',
                 'V': f'F{left_oct}',
                 'G': f'F#{left_oct}',
                 'B': f'G{left_oct}',
                 'H': f'G#{left_oct}',
                 'N': f'A{left_oct}',
                 'J': f'A#{left_oct}',
                 'M': f'B{left_oct}',
                 ',': f'C{left_oct + 1}'}

    timer.tick(fps)
    screen.fill('gray')
    white_keys, black_keys, active_whites, active_blacks = draw_piano(active_whites, active_blacks)
    draw_hands(left_oct, left_hand)
    draw_title_bar()
    for event in pygame.event.get():
        if event.type == pygame.QUIT: # 창을 닫으면 종료
            run = False
        if event.type == pygame.MOUSEBUTTONDOWN: #마우스 클릭
            black_key = False
            for i in range(len(black_keys)):
                if black_keys[i].collidepoint(event.pos):
                    black_sounds[i].play()  # 바로 재생
                    black_key = True
                    active_blacks.append([i, 30])
            for i in range(len(white_keys)):
                if white_keys[i].collidepoint(event.pos) and not black_key:
                    white_sounds[i].play()  # 바로 재생
                    active_whites.append([i, 30])
        if event.type == pygame.TEXTINPUT: #키보드로 문자 입력시
            if event.text.upper() in left_dict:
                if left_dict[event.text.upper()][1] == '#':
                    index = black_labels.index(left_dict[event.text.upper()])
                    black_sounds[index].play()  # 바로 재생
                    active_blacks.append([index, 30])
                else:
                    index = white_notes.index(left_dict[event.text.upper()])
                    white_sounds[index].play()  # 바로 재생
                    active_whites.append([index, 30])
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_UP:
                if left_oct < 9:
                    left_oct += 1
            if event.key == pygame.K_DOWN:
                if left_oct > 0:
                    left_oct -= 1


    pygame.display.flip()

pygame.quit()